In [1]:
import gurobipy as gp
from gurobipy import GRB

In [13]:
#Round1 Day 1
import pandas as pd
from gurobipy import Model, GRB

# Load the provided data files
problem_data = pd.read_csv("round1-day1_problem_data.csv")
truck_node_data = pd.read_csv("round1-day1_truck_node_data.csv")
demand_node_data = pd.read_csv("round1-day1_demand_node_data.csv")
demand_truck_data = pd.read_csv("round1-day1_demand_truck_data.csv")

# Extract relevant information
burrito_price = float(problem_data['burrito_price'][0])
ingredient_cost = float(problem_data['ingredient_cost'][0])
truck_cost = float(problem_data['truck_cost'][0])

truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in truck_node_data.iterrows()}
truck_spots = truck_coordinates.keys()

buildings = demand_node_data['index'].tolist()
building_names = demand_node_data['name'].tolist()
building_coordinates = {row['index']:(float(row['x']), float(row['y'])) for ind,row in demand_node_data.iterrows()}
demand = {row['index']:float(row['demand']) for ind,row in demand_node_data.iterrows()}

building_truck_spot_pairs = [(row['demand_node_index'], row['truck_node_index']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0]
distance = {(row['demand_node_index'], row['truck_node_index']):float(row['distance']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}
scaled_demand = {(row['demand_node_index'], row['truck_node_index']):float(row['scaled_demand']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}

# Declare and initialize model
model = Model("Burrito Optimization Game")

# Create decision variables for the Burrito Optimization Game model
x_placed = model.addVars(truck_spots, vtype=GRB.BINARY, name="x_placed")
y_served = model.addVars(building_truck_spot_pairs, vtype=GRB.BINARY, name="y_served")

# Create truck-must-exist constraints
cons1 = model.addConstrs((y_served[b,t] <= x_placed[t] 
                          for b,t in building_truck_spot_pairs), 
                          name = "Ensure_truck_spot_open_to_serve")

# Create only one truck per customers at building constraint
cons2 = model.addConstrs((sum(y_served[b,t] for t in truck_spots if (b,t) in building_truck_spot_pairs) <= 1 for b in buildings), 
                         name="Ensure_one_truck_per_customers_at_building")

# Objective: maximize total profit = burrito_revenue - total_truck_cost
burrito_revenue = sum((burrito_price - ingredient_cost)*scaled_demand[b,t]*y_served[b,t] for b,t in building_truck_spot_pairs)
total_truck_cost = truck_cost*sum(x_placed[t] for t in truck_spots)

model.setObjective(burrito_revenue - total_truck_cost, GRB.MAXIMIZE)

# Solve the model
model.optimize()

# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Objective Value: ${model.objVal}")
    
    # Print trucks placed
    placed_trucks = [t for t in truck_spots if x_placed[t].x > 0.5]
    print(f"Trucks Placed: {placed_trucks}")
    
    # Print buildings served by each truck
    for t in placed_trucks:
        served_buildings = [(b, scaled_demand[b,t]) for b in buildings if (b,t) in building_truck_spot_pairs and y_served[b,t].x > 0.5]
        print(f"Truck {t} serves: {served_buildings}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 163 rows, 164 columns and 438 nonzeros
Model fingerprint: 0x03a85e35
Variable types: 0 continuous, 164 integer (164 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 20 rows and 25 columns
Presolve time: 0.00s
Presolved: 143 rows, 139 columns, 378 nonzeros
Variable types: 0 continuous, 139 integer (139 binary)

Root relaxation: objective 8.850000e+02, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0  

In [14]:
#round 1 Day 2
import pandas as pd
from gurobipy import Model, GRB

# Load the provided data files
problem_data = pd.read_csv("round1-day2_problem_data.csv")
truck_node_data = pd.read_csv("round1-day2_truck_node_data.csv")
demand_node_data = pd.read_csv("round1-day2_demand_node_data.csv")
demand_truck_data = pd.read_csv("round1-day2_demand_truck_data.csv")

# Extract relevant information
burrito_price = float(problem_data['burrito_price'][0])
ingredient_cost = float(problem_data['ingredient_cost'][0])
truck_cost = float(problem_data['truck_cost'][0])

truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in truck_node_data.iterrows()}
truck_spots = truck_coordinates.keys()

buildings = demand_node_data['index'].tolist()
building_names = demand_node_data['name'].tolist()
building_coordinates = {row['index']:(float(row['x']), float(row['y'])) for ind,row in demand_node_data.iterrows()}
demand = {row['index']:float(row['demand']) for ind,row in demand_node_data.iterrows()}

building_truck_spot_pairs = [(row['demand_node_index'], row['truck_node_index']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0]
distance = {(row['demand_node_index'], row['truck_node_index']):float(row['distance']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}
scaled_demand = {(row['demand_node_index'], row['truck_node_index']):float(row['scaled_demand']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}

# Declare and initialize model
model = Model("Burrito Optimization Game")

# Create decision variables for the Burrito Optimization Game model
x_placed = model.addVars(truck_spots, vtype=GRB.BINARY, name="x_placed")
y_served = model.addVars(building_truck_spot_pairs, vtype=GRB.BINARY, name="y_served")

# Create truck-must-exist constraints
cons1 = model.addConstrs((y_served[b,t] <= x_placed[t] 
                          for b,t in building_truck_spot_pairs), 
                          name = "Ensure_truck_spot_open_to_serve")

# Create only one truck per customers at building constraint
cons2 = model.addConstrs((sum(y_served[b,t] for t in truck_spots if (b,t) in building_truck_spot_pairs) <= 1 for b in buildings), 
                         name="Ensure_one_truck_per_customers_at_building")

# Objective: maximize total profit = burrito_revenue - total_truck_cost
burrito_revenue = sum((burrito_price - ingredient_cost)*scaled_demand[b,t]*y_served[b,t] for b,t in building_truck_spot_pairs)
total_truck_cost = truck_cost*sum(x_placed[t] for t in truck_spots)

model.setObjective(burrito_revenue - total_truck_cost, GRB.MAXIMIZE)

# Solve the model
model.optimize()

# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Objective Value: ${model.objVal}")
    
    # Print trucks placed
    placed_trucks = [t for t in truck_spots if x_placed[t].x > 0.5]
    print(f"Trucks Placed: {placed_trucks}")
    
    # Print buildings served by each truck
    for t in placed_trucks:
        served_buildings = [(b, scaled_demand[b,t]) for b in buildings if (b,t) in building_truck_spot_pairs and y_served[b,t].x > 0.5]
        print(f"Truck {t} serves: {served_buildings}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 439 rows, 432 columns and 1218 nonzeros
Model fingerprint: 0xf29734af
Variable types: 0 continuous, 432 integer (432 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 439 rows, 432 columns, 1218 nonzeros
Variable types: 0 continuous, 432 integer (432 binary)

Root relaxation: objective 4.775000e+03, 249 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    4775.0000000 4775.00000  0.00%     

In [15]:
#Round 1 Day 3
import pandas as pd
from gurobipy import Model, GRB

# Load the provided data files
problem_data = pd.read_csv("round1-day3_problem_data.csv")
truck_node_data = pd.read_csv("round1-day3_truck_node_data.csv")
demand_node_data = pd.read_csv("round1-day3_demand_node_data.csv")
demand_truck_data = pd.read_csv("round1-day3_demand_truck_data.csv")

# Extract relevant information
burrito_price = float(problem_data['burrito_price'][0])
ingredient_cost = float(problem_data['ingredient_cost'][0])
truck_cost = float(problem_data['truck_cost'][0])

truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in truck_node_data.iterrows()}
truck_spots = truck_coordinates.keys()

buildings = demand_node_data['index'].tolist()
building_names = demand_node_data['name'].tolist()
building_coordinates = {row['index']:(float(row['x']), float(row['y'])) for ind,row in demand_node_data.iterrows()}
demand = {row['index']:float(row['demand']) for ind,row in demand_node_data.iterrows()}

building_truck_spot_pairs = [(row['demand_node_index'], row['truck_node_index']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0]
distance = {(row['demand_node_index'], row['truck_node_index']):float(row['distance']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}
scaled_demand = {(row['demand_node_index'], row['truck_node_index']):float(row['scaled_demand']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}

# Declare and initialize model
model = Model("Burrito Optimization Game")

# Create decision variables for the Burrito Optimization Game model
x_placed = model.addVars(truck_spots, vtype=GRB.BINARY, name="x_placed")
y_served = model.addVars(building_truck_spot_pairs, vtype=GRB.BINARY, name="y_served")

# Create truck-must-exist constraints
cons1 = model.addConstrs((y_served[b,t] <= x_placed[t] 
                          for b,t in building_truck_spot_pairs), 
                          name = "Ensure_truck_spot_open_to_serve")

# Create only one truck per customers at building constraint
cons2 = model.addConstrs((sum(y_served[b,t] for t in truck_spots if (b,t) in building_truck_spot_pairs) <= 1 for b in buildings), 
                         name="Ensure_one_truck_per_customers_at_building")

# Objective: maximize total profit = burrito_revenue - total_truck_cost
burrito_revenue = sum((burrito_price - ingredient_cost)*scaled_demand[b,t]*y_served[b,t] for b,t in building_truck_spot_pairs)
total_truck_cost = truck_cost*sum(x_placed[t] for t in truck_spots)

model.setObjective(burrito_revenue - total_truck_cost, GRB.MAXIMIZE)

# Solve the model
model.optimize()

# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Objective Value: ${model.objVal}")
    
    # Print trucks placed
    placed_trucks = [t for t in truck_spots if x_placed[t].x > 0.5]
    print(f"Trucks Placed: {placed_trucks}")
    
    # Print buildings served by each truck
    for t in placed_trucks:
        served_buildings = [(b, scaled_demand[b,t]) for b in buildings if (b,t) in building_truck_spot_pairs and y_served[b,t].x > 0.5]
        print(f"Truck {t} serves: {served_buildings}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 436 rows, 429 columns and 1209 nonzeros
Model fingerprint: 0xcffff52a
Variable types: 0 continuous, 429 integer (429 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 12 rows and 14 columns
Presolve time: 0.00s
Presolved: 424 rows, 415 columns, 1173 nonzeros
Variable types: 0 continuous, 415 integer (415 binary)

Root relaxation: objective 2.555000e+03, 268 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               

In [16]:
#round 1 Day 4
import pandas as pd
from gurobipy import Model, GRB

# Load the provided data files
problem_data = pd.read_csv("round1-day4_problem_data.csv")
truck_node_data = pd.read_csv("round1-day4_truck_node_data.csv")
demand_node_data = pd.read_csv("round1-day4_demand_node_data.csv")
demand_truck_data = pd.read_csv("round1-day4_demand_truck_data.csv")

# Extract relevant information
burrito_price = float(problem_data['burrito_price'][0])
ingredient_cost = float(problem_data['ingredient_cost'][0])
truck_cost = float(problem_data['truck_cost'][0])

truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in truck_node_data.iterrows()}
truck_spots = truck_coordinates.keys()

buildings = demand_node_data['index'].tolist()
building_names = demand_node_data['name'].tolist()
building_coordinates = {row['index']:(float(row['x']), float(row['y'])) for ind,row in demand_node_data.iterrows()}
demand = {row['index']:float(row['demand']) for ind,row in demand_node_data.iterrows()}

building_truck_spot_pairs = [(row['demand_node_index'], row['truck_node_index']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0]
distance = {(row['demand_node_index'], row['truck_node_index']):float(row['distance']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}
scaled_demand = {(row['demand_node_index'], row['truck_node_index']):float(row['scaled_demand']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}

# Declare and initialize model
model = Model("Burrito Optimization Game")

# Create decision variables for the Burrito Optimization Game model
x_placed = model.addVars(truck_spots, vtype=GRB.BINARY, name="x_placed")
y_served = model.addVars(building_truck_spot_pairs, vtype=GRB.BINARY, name="y_served")

# Create truck-must-exist constraints
cons1 = model.addConstrs((y_served[b,t] <= x_placed[t] 
                          for b,t in building_truck_spot_pairs), 
                          name = "Ensure_truck_spot_open_to_serve")

# Create only one truck per customers at building constraint
cons2 = model.addConstrs((sum(y_served[b,t] for t in truck_spots if (b,t) in building_truck_spot_pairs) <= 1 for b in buildings), 
                         name="Ensure_one_truck_per_customers_at_building")

# Objective: maximize total profit = burrito_revenue - total_truck_cost
burrito_revenue = sum((burrito_price - ingredient_cost)*scaled_demand[b,t]*y_served[b,t] for b,t in building_truck_spot_pairs)
total_truck_cost = truck_cost*sum(x_placed[t] for t in truck_spots)

model.setObjective(burrito_revenue - total_truck_cost, GRB.MAXIMIZE)

# Solve the model
model.optimize()

# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Objective Value: ${model.objVal}")
    
    # Print trucks placed
    placed_trucks = [t for t in truck_spots if x_placed[t].x > 0.5]
    print(f"Trucks Placed: {placed_trucks}")
    
    # Print buildings served by each truck
    for t in placed_trucks:
        served_buildings = [(b, scaled_demand[b,t]) for b in buildings if (b,t) in building_truck_spot_pairs and y_served[b,t].x > 0.5]
        print(f"Truck {t} serves: {served_buildings}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 86 rows, 79 columns and 159 nonzeros
Model fingerprint: 0x48c81560
Variable types: 0 continuous, 79 integer (79 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 70 rows and 64 columns
Presolve time: 0.00s
Presolved: 16 rows, 15 columns, 36 nonzeros
Found heuristic solution: objective 610.0000000
Variable types: 0 continuous, 15 integer (15 binary)

Root relaxation: objective 1.255000e+03, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [17]:
#Round 1 Day 5
import pandas as pd
from gurobipy import Model, GRB

# Load the provided data files
problem_data = pd.read_csv("round1-day5_problem_data.csv")
truck_node_data = pd.read_csv("round1-day5_truck_node_data.csv")
demand_node_data = pd.read_csv("round1-day5_demand_node_data.csv")
demand_truck_data = pd.read_csv("round1-day5_demand_truck_data.csv")

# Extract relevant information
burrito_price = float(problem_data['burrito_price'][0])
ingredient_cost = float(problem_data['ingredient_cost'][0])
truck_cost = float(problem_data['truck_cost'][0])

truck_coordinates = {row['index']:(float(row['x']),float(row['y'])) for ind,row in truck_node_data.iterrows()}
truck_spots = truck_coordinates.keys()

buildings = demand_node_data['index'].tolist()
building_names = demand_node_data['name'].tolist()
building_coordinates = {row['index']:(float(row['x']), float(row['y'])) for ind,row in demand_node_data.iterrows()}
demand = {row['index']:float(row['demand']) for ind,row in demand_node_data.iterrows()}

building_truck_spot_pairs = [(row['demand_node_index'], row['truck_node_index']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0]
distance = {(row['demand_node_index'], row['truck_node_index']):float(row['distance']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}
scaled_demand = {(row['demand_node_index'], row['truck_node_index']):float(row['scaled_demand']) for ind,row in demand_truck_data.iterrows() if float(row['scaled_demand'])>0}

# Declare and initialize model
model = Model("Burrito Optimization Game")

# Create decision variables for the Burrito Optimization Game model
x_placed = model.addVars(truck_spots, vtype=GRB.BINARY, name="x_placed")
y_served = model.addVars(building_truck_spot_pairs, vtype=GRB.BINARY, name="y_served")

# Create truck-must-exist constraints
cons1 = model.addConstrs((y_served[b,t] <= x_placed[t] 
                          for b,t in building_truck_spot_pairs), 
                          name = "Ensure_truck_spot_open_to_serve")

# Create only one truck per customers at building constraint
cons2 = model.addConstrs((sum(y_served[b,t] for t in truck_spots if (b,t) in building_truck_spot_pairs) <= 1 for b in buildings), 
                         name="Ensure_one_truck_per_customers_at_building")

# Objective: maximize total profit = burrito_revenue - total_truck_cost
burrito_revenue = sum((burrito_price - ingredient_cost)*scaled_demand[b,t]*y_served[b,t] for b,t in building_truck_spot_pairs)
total_truck_cost = truck_cost*sum(x_placed[t] for t in truck_spots)

model.setObjective(burrito_revenue - total_truck_cost, GRB.MAXIMIZE)

# Solve the model
model.optimize()

# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Objective Value: ${model.objVal}")
    
    # Print trucks placed
    placed_trucks = [t for t in truck_spots if x_placed[t].x > 0.5]
    print(f"Trucks Placed: {placed_trucks}")
    
    # Print buildings served by each truck
    for t in placed_trucks:
        served_buildings = [(b, scaled_demand[b,t]) for b in buildings if (b,t) in building_truck_spot_pairs and y_served[b,t].x > 0.5]
        print(f"Truck {t} serves: {served_buildings}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.3.0 24D70)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1113 rows, 1115 columns and 3177 nonzeros
Model fingerprint: 0x4b92d318
Variable types: 0 continuous, 1115 integer (1115 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.01s
Presolved: 1113 rows, 1115 columns, 3177 nonzeros
Variable types: 0 continuous, 1115 integer (1115 binary)

Root relaxation: objective 7.850000e+03, 581 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7850.0000000 7850.00000  0.